In [212]:
import pandas as pd
import xml.etree.ElementTree as ET

# Leer archivo Excel
df = pd.read_excel("./salida/demo.xlsx")

# Vista previa para confirmar columnas

df.head()


,payloadID,timestamp,version,from_credential_domain,from_identity,from_correspondent_name,to_credential1_domain,to_credential1_identity,to_credential2_domain,to_credential2_identity,...,tax_description,tax_category,tax_percentageRate,tax_taxable_amount,tax_taxable_currency,tax_amount,tax_amount_currency,net_amount,net_currency,isHeaderInvoice
0,UBSRSO20250624T122427658,2025-06-24 08:24:27,12045,NetworkId,AN11183544707,London Stock Exchange Plc,NetworkID,AN01395405443,SystemID,Default,...,Total Tax,vat,1750.0,848791.0,GBP,148500.0,GBP,997291.0,GBP,NaN
1,UBSRSO20250624T1224276581,45832.353831,12045,NetworkId,AN11183544707,London Stock Exchange Plc,NetworkID,AN01395405443,SystemID,Default,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
df.columns

Index(['payloadID', 'timestamp', 'version', 'from_credential_domain',
       'from_identity', 'from_correspondent_name', 'to_credential1_domain',
       'to_credential1_identity', 'to_credential2_domain',
       'to_credential2_identity', 'sender_credential_domain',
       'sender_identity', 'sender_shared_secret', 'user_agent', 'request_id',
       'request_deploymentMode', 'header_invoiceDate', 'header_invoiceID',
       'header_invoiceOrigin', 'header_operation', 'header_purpose',
       'partner1_role', 'partner1_addressID', 'partner1_name',
       'partner1_email', 'partner2_role', 'partner2_addressID',
       'partner2_name', 'partner2_email', 'partner3_role',
       'partner3_addressID', 'partner3_name', 'partner3_email',
       'partner4_role', 'partner4_addressID', 'partner4_name',
       'partner4_email', 'paymentTerm_days', 'comments',
       'extrinsic_buyerVatID', 'extrinsic_supplierVatID',
       'extrinsic_invoicePDF', 'extrinsic_IBAN',
       'extrinsic_Bank Account Num

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import unicodedata

import pandas as pd
import xml.etree.ElementTree as ET

# Leer archivo Excel
df = pd.read_excel("./salida/demo.xlsx")

# Vista previa para confirmar columnas

df.head()

def _normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    def norm(s):
        s = unicodedata.normalize("NFKC", str(s))
        return s.strip()
    df = df.copy()
    df.columns = [norm(c) for c in df.columns]
    return df

def _find_col(df: pd.DataFrame, candidates):
    """Devuelve el nombre REAL de la primera columna que exista (match case-insensitive)."""
    cmap = {c.lower(): c for c in df.columns}
    for cand in candidates:
        c = cand.strip().lower()
        if c in cmap:
            return cmap[c]
    return None

def _first_value(g: pd.DataFrame, candidates, default=None):
    col = _find_col(g, candidates)
    if not col:
        return default
    s = g[col].dropna()
    return s.iloc[0] if not s.empty else default

def _iso_dt(val, default=None):
    if pd.isna(val):
        return default
    try:
        # Soporta dd/mm/yyyy hh:mm del Excel
        ts = pd.to_datetime(val, dayfirst=True, errors="coerce")
        if pd.isna(ts): 
            return default
        # Si viene solo fecha, añadimos T00:00:00
        return ts.isoformat()
    except Exception:
        return default


In [ ]:
def create_cxml_invoices(df: pd.DataFrame, group_field=None, output_prefix="invoice_"):
    df = _normalize_cols(df)

    # Aliases por campo (pon aquí nuevas variantes si tu Excel cambia)
    ALIAS = {
        "invoice_id":       ["invoiceid", "header_invoiceid", "invoice_id"],
        "invoice_date":     ["invoicedate", "header_invoicedate", "invoice_date"],
        "invoice_origin":   ["invoiceorigin", "header_invoiceorigin"],
        "operation":        ["operation", "header_operation"],
        "purpose":          ["purpose", "header_purpose"],

        "qty":              ["quantity", "qty", "cantidad"],
        "uom":              ["uom", "unitofmeasure", "unidad_medida"],
        "currency":         ["currency", "moneda"],
        "unit_price":       ["unitprice", "price", "unit_price", "precio_unitario"],
        "description":      ["description", "desc", "itemdescription", "descripcion"],
        "subtotal":         ["subtotal", "linetotal", "lineamount", "importe_linea"],
        "tax":              ["tax", "taxamount", "impuesto"],
        'partner1_addressID': ["partner1_addressid", "partner1_addressID", "partner1_address", "partner1"],
        'partner1_role': ["partner1_role"],
        'partner2_role': ["partner2_role"],
        'from_correspondent_name': ["from_correspondent_name"],
        'partner2_addressID': ["partner2_addressid", "partner2_addressID", "partner2_address", "partner2"],
        'comments': ["comments", "comentarios"],
    }

    # Detectar columna de agrupación si no se pasa
    if group_field is None:
        group_col = _find_col(df, ALIAS["invoice_id"])
    else:
        group_col = _find_col(df, [group_field])
        if group_col is None:
            # si el nombre dado no existe, intenta por alias
            group_col = _find_col(df, ALIAS["invoice_id"])

    # Si no hay campo de agrupación, todo es 1 factura
    if not group_col:
        groups = [(None, df)]
    else:
        groups = df.groupby(group_col)

    for inv_id, g in groups:
        g = g.reset_index(drop=True)

        # Raíz cXML
        cxml = ET.Element("cXML", attrib={
            "payloadID": f"auto_{pd.Timestamp.now().timestamp()}",
            "timestamp": pd.Timestamp.now().isoformat(),
            "version": "1.2.045",
        })
        request = ET.SubElement(cxml, "Request", attrib={"Id": "cXMLData", "deploymentMode": "test"})
        inv_req = ET.SubElement(request, "InvoiceDetailRequest")

        # Header 100% dinámico
        invoice_id   = str(inv_id) if inv_id is not None else str(_first_value(g, ALIAS["invoice_id"], "INV001"))
        inv_date_raw = _first_value(g, ALIAS["invoice_date"], pd.Timestamp.today())
        invoice_date = _iso_dt(inv_date_raw, pd.Timestamp.today().isoformat())
        invoice_origin = str(_first_value(g, ALIAS["invoice_origin"], "supplier"))
        operation      = str(_first_value(g, ALIAS["operation"], "new"))
        purpose        = str(_first_value(g, ALIAS["purpose"], "standard"))

        hdr = ET.SubElement(inv_req, "InvoiceDetailRequestHeader", attrib={
            "invoiceDate": invoice_date,
            "invoiceID": invoice_id,
            "invoiceOrigin": invoice_origin,
            "operation": operation,
            "purpose": purpose,
        })

        # @attr isTaxInLine Set to "yes" if tax info (Tax) is provided at invoice line level. 
        # @attr isSpecialHandlingInLine Set to "yes" if special handling info (InvoiceDetailLineSpecialHandling) is provided at invoice line level.
        # @attr isShippingInLine Set to "yes" if shipping info (InvoiceDetailLineShipping) is provided at invoice line level. 
        # @attr isDiscountInLine Set to "yes" if discount info (InvoiceDetailDiscount) is provided at invoice line level. 
        # @attr isAccountingInLine Set to "yes" if accounting distribution info (Distribution) is provided at invoice line level. If isHeaderInvoice is true, this indicator must be false because Distribution is only available at item level.
        # @attr isPriceAdjustmentInLine Set to "yes" if a line-item credit memo or debit memo has a price adjustment.
        # I have to be sure of how this works

        


        InvoiceDetailLineIndicator = ET.SubElement(hdr, "InvoiceDetailLineIndicator",attrib={"isTaxInLine": "Yes"}) 
        # Items
        # Defines the invoice information of an order or master agreement with item details, 
        # used ONLY when isHeaderInvoice is false. 
        # In this case, an invoice line is an InvoiceDetailItem and its invoice line number is InvoiceDetailItem@invoiceLineNumber. 
        # InvoiceDetailOrderInfo: The reference information of the order or master agreement. InvoiceDetailReceiptInfo: 
        # The reference information of the receipt. InvoiceDetailShipNoticeInfo: The reference information of the ship notice. 
        # InvoiceDetailItem: The information of an item in the order. 
        # InvoiceDetailServiceItem: The information of a service item in the master agreement.




        # Defines a party involved in invoicing, including Issuer Of Invoice (issuerOfInvoice), S
        # old to (soldTo), Bill to (billTo), Remit to (remitTo), etc. 
        # This element should NOT be used for specifying ship from and ship to.
        # For that purpose, InvoiceDetailShipping should be used. 
        # Contact: Contact information of the invoice partner.
        # Allowed contact roles include issuerOfInvoice, soldTo, billTo, and remitTo.
        # IdReference: Partner reference ID, including account #, 
        # ERP account payable ID, ERP account receivable ID, bank routing ID 
        # for remittance, supplier tax ID, VAT ID, 1099 ID, court register ID, tax exemption ID, and so on.


        # remitTo, billTo, soldTo, from

        # invoicePartner -- remitTo
        InvoicePartner = ET.SubElement(hdr, "InvoicePartner")
        Contact = ET.SubElement(InvoicePartner, "Contact", attrib={"addressID": _first_value(g,ALIAS['partner1_addressID']), 'role': _first_value(g,ALIAS['partner1_role'])})
        name = ET.SubElement(Contact, "Name", attrib={"xml:lang": ""})
        
        # invoicePartner -- From
        InvoicePartner = ET.SubElement(hdr, "InvoicePartner")
        Contact = ET.SubElement(InvoicePartner, "Contact",
                                 attrib={"addressID": _first_value(g,ALIAS['partner2_addressID'],""), 'role': _first_value(g,ALIAS['partner2_role'])})
        name = ET.SubElement(Contact, "Name", attrib={"xml:lang":""}).text = _first_value(g,ALIAS['from_correspondent_name'],"")


        # invoicePartner -- From
        InvoicePartner = ET.SubElement(hdr, "InvoicePartner")
        Contact = ET.SubElement(InvoicePartner, "Contact",
                                 attrib={"addressID": _first_value(g,ALIAS['partner2_addressID']), 'role': _first_value(g,ALIAS['partner2_role'])})
        name = ET.SubElement(Contact, "Name", attrib={"xml:lang":""}).text = _first_value(g,ALIAS['from_correspondent_name'],"")

        # invoicePartner -- billTo
        InvoicePartner = ET.SubElement(hdr, "InvoicePartner")
        Contact = ET.SubElement(InvoicePartner, "Contact",
                                 attrib={"addressID": _first_value(g,ALIAS['partner2_addressID']), 'role': _first_value(g,ALIAS['partner2_role'])})
        name = ET.SubElement(Contact, "Name", attrib={"xml:lang":""}).text = _first_value(g,ALIAS['from_correspondent_name'],"")

        # commments = ET.SubElement(Contact, "Comments").text = _first_value(g, ALIAS['comments'], "")
        # @attr identifier The unique identifier of the IdReference within the domain. @attr domain The domain of the IdReference. Supported values depend on your procurement application. Possible values:

        # 1099ID
        # abaRoutingNumber
        # accountID
        # accountPayableID
        # accountReceivableID
        # bankAccountID
        # bankBranchID
        # bankNationalID
        # bankRoutingID
        # buyerLocationID
        # companyRegistrationNumber
        # courtRegisterID
        # creditorRefID
        # documentName
        # federalTaxID
        # governmentNumber
        # gstID
        # ibanID
        # isoBicID
        # iso6523
        # provincialTaxID
        # stateTaxID
        # storageLocationID
        # supplierReference
        # swiftID
        # taxExemptionID
        # unloadingPointID
        # vatID
        # ID
        # Note: supplierTaxID is deprecated and will be treated as federalTaxID.
        IdReference = ET.SubElement(Contact, "IdReference", attrib={"domain": "DUNS"})
       
       
       
       
       
        order = ET.SubElement(inv_req, ""
        "")
        for i, row in g.iterrows():
            qty   = row.get(_find_col(g, ALIAS["qty"]), 1)
            uom   = row.get(_find_col(g, ALIAS["uom"]), "EA")
            curr  = row.get(_find_col(g, ALIAS["currency"]), "USD")
            price = row.get(_find_col(g, ALIAS["unit_price"]), 0)
            desc  = row.get(_find_col(g, ALIAS["description"]), "")
            sub   = row.get(_find_col(g, ALIAS["subtotal"]), 0)

            item = ET.SubElement(order, "InvoiceDetailItem", attrib={
                "invoiceLineNumber": str(i + 1),
                "quantity": str(qty),
            })
            ET.SubElement(item, "UnitOfMeasure").text = str(uom)

            unit_price = ET.SubElement(item, "UnitPrice")
            ET.SubElement(unit_price, "Money", attrib={"currency": str(curr)}).text = str(price)

            ref = ET.SubElement(item, "InvoiceDetailItemReference", attrib={"lineNumber": str(i + 1)})
            ET.SubElement(ref, "Description", attrib={"xml:lang": "en"}).text = str(desc)

            subtotal = ET.SubElement(item, "SubtotalAmount")
            ET.SubElement(subtotal, "Money", attrib={"currency": str(curr)}).text = str(sub)

        # Summary (valida moneda única por factura)
        summary = ET.SubElement(inv_req, "InvoiceDetailSummary")
        cur_col = _find_col(g, ALIAS["currency"])
        if cur_col:
            currencies = g[cur_col].dropna().astype(str).unique().tolist()
        else:
            currencies = ["USD"]

        if len(currencies) > 1:
            raise ValueError(f"La factura {invoice_id} tiene varias monedas: {currencies}. "
                             "Un cXML válido requiere una sola moneda por factura.")

        currency = currencies[0]
        subtotal_total = g.get(_find_col(g, ALIAS["subtotal"]), pd.Series([0]*len(g))).sum()
        tax_total      = g.get(_find_col(g, ALIAS["tax"]), pd.Series([0]*len(g))).sum()

        sub_el = ET.SubElement(summary, "SubtotalAmount")
        ET.SubElement(sub_el, "Money", attrib={ "alternateCurrency": currency,"currency": currency }).text = str(subtotal_total)

        tax_el = ET.SubElement(summary, "Tax")
        ET.SubElement(tax_el, "Money", attrib={"currency": currency}).text = str(tax_total)

        net_el = ET.SubElement(summary, "NetAmount")
        ET.SubElement(net_el, "Money", attrib={"currency": currency}).text = str(subtotal_total + tax_total)

        # Guardar
        out_name = f"{output_prefix}{invoice_id}.xml"
        ET.ElementTree(cxml).write(out_name, encoding="utf-8", xml_declaration=True)
        print(f"✅ XML generado: {out_name}")


In [216]:
# df = pd.read_excel("demo.xlsx", sheet_name=0)
create_cxml_invoices(df, group_field=None, output_prefix="./salida/invoice_")


✅ XML generado: ./salida/invoice_4701265854.xml
